In [1]:
from myenv.models.candlestick import Candlestick
from myenv.models.merchandise_rate import MerchandiseRate
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


merchandise_rate = MerchandiseRate()
merchandise_rate_id = merchandise_rate.find_by_slug('BTCUSDT')
candlestick = Candlestick(merchandise_rate_id, 'hour')


bitcoin_prices = candlestick.to_df()

bitcoin_prices.describe()

bitcoin_prices['hour return'] = bitcoin_prices['close'].pct_change() * 100

bitcoin_prices.tail(20)


,date,open,high,close,low,hour return
36788,2021-10-03 22:00:00+07:00,47965.6,48025.3,47879.8,47799.2,-0.178878
36789,2021-10-03 23:00:00+07:00,47879.8,48067.7,48035.0,47822.0,0.324145
36790,2021-10-04 00:00:00+07:00,48035.0,49228.1,48910.2,47943.1,1.822005
36791,2021-10-04 01:00:00+07:00,48910.2,49176.3,48822.0,48541.3,-0.180330
36792,2021-10-04 02:00:00+07:00,48822.0,48961.0,48774.8,48725.0,-0.096678
36793,2021-10-04 03:00:00+07:00,48774.8,48837.9,48425.0,48400.6,-0.717174
36794,2021-10-04 04:00:00+07:00,48425.0,48455.1,47758.2,47601.9,-1.376975
36795,2021-10-04 05:00:00+07:00,47758.2,48200.0,47984.9,47700.0,0.474683
36796,2021-10-04 06:00:00+07:00,47988.2,48320.0,48200.0,47874.0,0.448266
36797,2021-10-04 07:00:00+07:00,48200.0,48280.7,47860.9,47840.0,-0.703527
